# Auswertung der Parquet Dateien aus dem Echtzeitarchiv V14

## Import der Module und Setzen Parameter

In [1]:
import duckdb
import pandas as pd
import seaborn as sns
import sys
import openpyxl

import datetime as dt
import importlib

from dotenv import load_dotenv, dotenv_values
import logging

In [2]:
log_file = f"log/log_rt.txt"
logging.basicConfig(filename=log_file, 
                        level=logging.DEBUG,
                        style="{",
                        format="{asctime} [{levelname:8}] {message}",
                        datefmt="%d.%m.%Y %H:%M:%S")

load_dotenv()

True

In [3]:
sys.path.append('/home/zvbn/python/rt2')

In [4]:
from class_rt_duck import rt_duck

In [5]:
logging.info("Auswertung RT aus parquet gestartet")

In [6]:
config = dotenv_values(".env")
#config

In [7]:
pd.options.display.max_columns = 100

In [8]:
jetzt = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
gestern= (dt.date.today() - dt.timedelta(1)).strftime('%Y-%m-%d')
letzte07tage= (dt.date.today() - dt.timedelta(7)).strftime('%Y-%m-%d')
letzte14tage= (dt.date.today() - dt.timedelta(14)).strftime('%Y-%m-%d')
letzte21tage= (dt.date.today() - dt.timedelta(21)).strftime('%Y-%m-%d')

print(jetzt, letzte21tage)

2025-01-10 16:28:12 2024-12-20


## Funktionen

In [9]:
def replace_german_special_characters(text):
    replacements = {
        'ä': 'ae',
        'ö': 'oe',
        'ü': 'ue',
        'Ä': 'Ae',
        'Ö': 'Oe',
        'Ü': 'Ue',
        'ß': 'ss'
    }
    
    for german_char, replacement in replacements.items():
        text = text.replace(german_char, replacement)
    
    return text

### Testen der class

In [10]:
rt = rt_duck()
rt

In [11]:
#Schließen der Verbindung
#rt.verbindung_schließen()

In [12]:
rt.create_table_fahrten(server = 'prod')

Table 'fahrten' created.


In [13]:
rt.create_table_zusatz(server = 'prod')
rt.create_table_verlauf(server = 'prod')
rt.create_table_matrix(server = 'prod')

Table 'zusatz' created.
Table 'verlauf' created.
Table 'matrix' created.


In [28]:
rt.cursor.sql("select distinct lineid from fahrten")

┌───────────────┐
│    lineid     │
│    varchar    │
├───────────────┤
│ de:VBN:N9:    │
│ de:VBN:722:1  │
│ de:VBN:24:    │
│ de:VBN:RB37:  │
│ de:VBN:33:    │
│ de:VBN:611:   │
│ de:VBN:114:   │
│ de:VBN:80:1   │
│ de:VBN:20:1   │
│ de:VBN:640:   │
│      ·        │
│      ·        │
│      ·        │
│ de:VBN:419:   │
│ de:VBN:784:   │
│ de:VBN:344:2  │
│ de:VBN:1E:601 │
│ de:VBN:380:3  │
│ de:VBN:N38:   │
│ de:VBN:66:    │
│ de:VBN:N73:2  │
│ de:VBN:608:   │
│ de:VBN:777:   │
├───────────────┤
│   744 rows    │
│  (20 shown)   │
└───────────────┘

In [50]:
rt.cursor.sql(""" pivot(
              select datum::date as datum, buendel, 
              round((count (*) filter (realtimeHasEverBeenReported = true) / count(*) ) * 100, 2) as anteil_ez
  
              from fahrten f 
                join linien l on f.lineid_short = l.dlid 
              where 
               f.datum > (current_date - interval 60 day)
              and ebene in ('1+', '1', '2', 'Stadt')
              group by all)
              on buendel
              using sum(anteil_ez)
              order by datum desc""").df().style.background_gradient(cmap=sns.diverging_palette(0, 120, as_cmap=True)).to_html('/var/www/rt_archiv/log_12_pivot_neu.html', encoding='LATIN1')

In [23]:
rt.anzahl_fahrten_betreiber()

,vu,count_ges,count_rt,heute_minus_1_ges,heute_minus_1_rt,anteil_heute_minus_1,heute_minus_2_ges,heute_minus_2_rt,anteil_heute_minus_2,heute_minus_3_ges,heute_minus_3_rt,anteil_heute_minus_3
0,Bremer Straßenbahn AG,657404,286667,5813,5729,98.55%,5832,5729,98.23%,5800,5725,98.71%
1,Verkehr und Wasser GmbH (VWG),237364,95193,2124,2115,99.58%,2122,2113,99.58%,2120,2086,98.40%
2,BREMERHAVEN BUS,156174,41884,1336,857,64.15%,1336,855,64.00%,1336,847,63.40%
3,KVG Stade GmbH & Co. KG,133433,4881,1074,134,12.48%,1074,139,12.94%,1074,141,13.13%
4,Verkehrsbetriebe Oldenburger Land,125606,46361,1457,885,60.74%,1456,1204,82.69%,1464,1228,83.88%
5,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,68358,10771,781,275,35.21%,782,282,36.06%,783,282,36.02%
6,NordWestBahn,55552,23880,437,433,99.08%,437,437,100.00%,503,499,99.20%
7,Delbus GmbH & Co. KG,53264,21674,453,424,93.60%,453,434,95.81%,453,398,87.86%
8,Reisedienst von Rahden GmbH & Co. KG,46358,18228,578,529,91.52%,581,520,89.50%,580,525,90.52%
9,Weser-Ems-Bus Betrieb Bremen,44637,11402,550,330,60.00%,549,321,58.47%,549,329,59.93%


In [20]:
rt.anzahl_fahrten_betreiber().to_html('/var/www/rt_archiv/anzahl_fahrten_betreiber.html', encoding='LATIN1')

## Ermitteln der Fahrten, die nur 0 Min senden

In [15]:
rt.cursor.sql("""select ex_lineid, fnr,min(operday) as start, max(operday) as ende ,count(*) as count from (
                select * from 
                    (select operday, ex_lineid, fnr, avg(dep_del) as avg_del
                    from verlauf 
                    where dep_del is not null
                    and has_rt = true
                    group by all)
                where avg_del = 0 and ex_lineid like 'de:VBN:6__:%'
                order by ex_lineid) 
              
              group by all
              order by count desc
              """).df().to_excel('out/verlauf_0min.xlsx', index=False)

In [16]:
rt.cursor.sql("select * from verlauf where fnr = '1630018'").df().to_excel('out/verlauf_1630018.xlsx', index=False)

In [17]:
suffix = 'mitte'
#auswahl_linien = '680|660|N68'
auswahl_linien = '630|670|N63|N67'

df_auswahl_ohne_rt = rt.cursor.sql(f"""select * from
              (select lineshort, min(datum)::date as min_datum, max(datum)::date as max_datum, fnr, 
              count(* ) as anzahl, 
              count(* ) filter (hasRealtime  = false ) as anzahl_ohne_rt, 
              anzahl_ohne_rt / count(* ) as proz_ohne_rt
              from fahrten  
              where lineid  SIMILAR TO '.*({auswahl_linien}).*' 
              -- and hasRealtime  = false 
              and datum  >= (current_date() - interval 28 days)
              group by all
              )
              where anzahl_ohne_rt > 1
              order by proz_ohne_rt desc

              -- limit 10""").df()

df_zusatz = rt.cursor.sql(f"""select * from zusatz where lineid  SIMILAR TO 'de:VBN:.*({auswahl_linien}).*' """).df()

ohne_rt_xl = f"out/rt_ohne_realtime_{suffix}.xlsx"
sn01 = 'ohne_rt'
with pd.ExcelWriter(ohne_rt_xl, engine='openpyxl') as writer:
    df_auswahl_ohne_rt.to_excel(writer, sheet_name=sn01, index=False)
    worksheet = writer.sheets[sn01]
    worksheet.freeze_panes = 'a2'

    worksheet.column_dimensions['B'].width = 15
    worksheet.column_dimensions['C'].width = 15
    worksheet.auto_filter.ref = worksheet.dimensions

    # Format the 'Zeit' column as date
    for cell in worksheet['B']:  # Assuming 'Zeit' is in column D
        if cell.row == 1:  # Skip the header row
            continue
        cell.number_format = 'YYYY-MM-DD'

    # Format the 'Zeit' column as date
    for cell in worksheet['C']:  # Assuming 'Zeit' is in column D
        if cell.row == 1:  # Skip the header row
            continue
        cell.number_format = 'YYYY-MM-DD'

    # Format the 'Prozent' column as percentage
    for cell in worksheet['G']:  # Assuming 'Prozent' is in column D
        if cell.row == 1:  # Skip the header row
            continue
        cell.number_format = '0.0%'
 
df_zusatz

,datum,fnr,destination,hasRealtime,vu,lineid,lineid_short,lineshort,reported_cancelled,journey_cancelled,ts_reported_cancelled,cancelled_kum,deviceid,clientid,journeyrttype,fahrtstarttime,fahrtstartstationname,fahrtstartstationdhid,fahrtendtime,fahrtendstationname,fahrtendstationdhid,realtimeHasEverBeenReported,filename
0,2024-09-01,1630226,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0901-1630226#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-01T20:00:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-01T21:29:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,NaN,out/parquet/prod/zusatz_2024_09_01.parquet
1,2024-09-02,1630054,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0902-1630054#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-02T18:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-02T19:56:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,NaN,out/parquet/prod/zusatz_2024_09_02.parquet
2,2024-09-02,1630058,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0902-1630058#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-02T20:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-02T21:43:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,NaN,out/parquet/prod/zusatz_2024_09_02.parquet
3,2024-09-09,1630054,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0909-1630054#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-09T18:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-09T19:56:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,False,out/parquet/prod/zusatz_2024_09_09.parquet
4,2024-09-09,1630058,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0909-1630058#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-09T20:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-09T21:43:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,False,out/parquet/prod/zusatz_2024_09_09.parquet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,2025-01-06,1670012,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670012#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T08:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T09:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet
281,2025-01-06,1670014,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670014#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T09:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T10:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet
282,2025-01-06,1670016,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670016#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T10:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T11:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet
283,2025-01-06,1670018,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670018#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T11:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T12:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet


In [18]:
df_linien_quote_rt = rt.cursor.sql(f"""select * from
              (select lineshort, min(datum)::date as min_datum, max(datum)::date as max_datum,  
              count(* ) as anzahl, 
              count(* ) filter (hasRealtime  = false ) as anzahl_ohne_rt, 
              anzahl_ohne_rt / count(* ) as proz_ohne_rt
              from fahrten  
                                   
                                   
             where 
              -- and hasRealtime  = false 
              datum  >= (current_date() - interval 28 days)
              group by all
              )
              where anzahl_ohne_rt > 1
              order by proz_ohne_rt desc

              -- limit 10""").df()

df_linien_quote_rt

,lineshort,min_datum,max_datum,anzahl,anzahl_ohne_rt,proz_ohne_rt
0,184,2024-12-12,2025-01-08,120,120,1.000000
1,374,2024-12-12,2025-01-08,40,40,1.000000
2,583,2024-12-12,2025-01-08,78,78,1.000000
3,842,2024-12-12,2025-01-08,130,130,1.000000
4,559,2024-12-12,2025-01-08,228,228,1.000000
...,...,...,...,...,...,...
514,6,2024-12-12,2025-01-08,5930,10,0.001686
515,503,2024-12-12,2025-01-08,1878,3,0.001597
516,24,2024-12-12,2025-01-08,5113,6,0.001173
517,21,2024-12-12,2025-01-08,3110,3,0.000965


In [19]:
df_zusatz

,datum,fnr,destination,hasRealtime,vu,lineid,lineid_short,lineshort,reported_cancelled,journey_cancelled,ts_reported_cancelled,cancelled_kum,deviceid,clientid,journeyrttype,fahrtstarttime,fahrtstartstationname,fahrtstartstationdhid,fahrtendtime,fahrtendstationname,fahrtendstationdhid,realtimeHasEverBeenReported,filename
0,2024-09-01,1630226,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0901-1630226#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-01T20:00:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-01T21:29:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,NaN,out/parquet/prod/zusatz_2024_09_01.parquet
1,2024-09-02,1630054,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0902-1630054#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-02T18:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-02T19:56:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,NaN,out/parquet/prod/zusatz_2024_09_02.parquet
2,2024-09-02,1630058,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0902-1630058#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-02T20:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-02T21:43:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,NaN,out/parquet/prod/zusatz_2024_09_02.parquet
3,2024-09-09,1630054,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0909-1630054#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-09T18:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-09T19:56:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,False,out/parquet/prod/zusatz_2024_09_09.parquet
4,2024-09-09,1630058,Zeven Bahnhof Süd,True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:630:,de:VBN:630,630,False,False,,False,0909-1630058#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2024-09-09T20:20:00+02:00,Bremen Hauptbahnhof,de:04011:13925::G,2024-09-09T21:43:00+02:00,Zeven Bahnhof Süd,de:03357:102760::7,False,out/parquet/prod/zusatz_2024_09_09.parquet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,2025-01-06,1670012,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670012#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T08:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T09:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet
281,2025-01-06,1670014,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670014#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T09:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T10:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet
282,2025-01-06,1670016,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670016#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T10:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T11:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet
283,2025-01-06,1670018,Osterholz-Scharmbeck Bahnhof (Bus),True,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,de:VBN:670:,de:VBN:670,670,False,False,,False,0106-1670018#!ADD!#IVU-Regio#,IVU-Regio,DEVIATION_OF_SCHEDULED,2025-01-06T11:40:00+01:00,Bremen Hauptbahnhof,de:04011:13925::G,2025-01-06T12:45:00+01:00,Osterholz-Scharmbeck Bahnhof (Bus),de:03356:72063::1,True,out/parquet/prod/zusatz_2025_01_06.parquet


### Auswertung Matrix nach Verlauf Zeitpunkt der Meldung

In [20]:
df_matrix = rt.cursor.sql("""select m.operatingDay::date, m.lineShortName, m.journeyId, v.index, 
                          m.stationName, m.scheduleDeparture,m.delay_minutes_arrival, m.delay_minutes_departure, m.timestamp, v.arr_del, v.dep_del
                from matrix m
                left join verlauf v on 
                          m.operatingDay = v.operday and 
                          m.lineShortName = v.lineshortname and 
                          m.journeyId = v.fnr and 
                          m.stationName = v.station_name
                where stop_cancelled = false
              and m.lineShortName = 'RS3'
              order by  m.operatingDay, m.externalLineId, m.journeyId, v.index,  m.timestamp 
              
              -- limit 20""").df()

In [21]:
auswahl_linien = '630|670|N68|N63|N67'
df_zusatz = rt.cursor.sql(f"""
                select datum::date as datum, lineshort,lineid ,fnr,  vu 
                from zusatz 
                where                       

                    lineid SIMILAR TO 'de:VBN:.*({auswahl_linien}).*' and 
                    -- and vu like 'Reisedienst von Rahden%' 
                    datum::date >= (current_date - interval 30 day)
                group by all 
                order by lineshort, fnr """).df()

df_zusatz

#rt.cursor.sql(f"""select * from zusatz where lineid  SIMILAR TO 'de:VBN:.*({auswahl_linien}).*' and datum::date >= (current_date - interval 30 day)""").df()

,datum,lineshort,lineid,fnr,vu
0,2025-01-07,630,de:VBN:630:,1630014,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
1,2024-12-23,630,de:VBN:630:,1630050,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
2,2024-12-18,630,de:VBN:630:,1630050,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
3,2024-12-19,630,de:VBN:630:,1630050,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
4,2024-12-20,630,de:VBN:630:,1630050,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
...,...,...,...,...,...
231,2024-12-29,670,de:VBN:670:,1670228,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
232,2024-12-22,670,de:VBN:670:,1670228,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
233,2024-12-29,670,de:VBN:670:,1670230,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH
234,2025-01-05,670,de:VBN:670:,1670230,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH


In [23]:
rt.cursor.sql("select min(datum )::date as min_date, max(datum)::date as amx_date, count(*) as anzahl from fahrten")

┌────────────┬────────────┬─────────┐
│  min_date  │  amx_date  │ anzahl  │
│    date    │    date    │  int64  │
├────────────┼────────────┼─────────┤
│ 2024-08-29 │ 2025-01-08 │ 1889943 │
└────────────┴────────────┴─────────┘

In [24]:
rt.create_vw_buendel('TN 5 CUX')

In [25]:
rt.cursor.sql("select * from vw_buendel").df()

,datum,buendel,ebene,vu,fnr,fahrtstartstationname,fahrtendstationname,lineshort,lineid_short,hasRealtime,journey_cancelled,reported_cancelled,ts_reported_cancelled,realtimeHasEverBeenReported
0,2024-10-28,TN 5 CUX,None,Reisedienst von Rahden GmbH & Co. KG,1545103,Flögeln Stüh,Bad Bederkesa Schulzentrum,545,de:VBN:545,True,False,False,,False
1,2024-10-28,TN 5 CUX,None,MEW Mobilitätszentrale Elbe-Weser,1555102,Drangstedt Grundschule,Elmlohe Grundschule,555,de:VBN:555,True,False,False,,False
2,2024-10-28,TN 5 CUX,None,MEW Mobilitätszentrale Elbe-Weser,1542101,Geestenseth(Schiffdorf) Bahnhof,Bad Bederkesa Moor-Therme,542,de:VBN:542,True,False,False,,False
3,2024-10-28,TN 5 CUX,None,MEW Mobilitätszentrale Elbe-Weser,1525101,Bad Bederkesa Moor-Therme,Bremerhaven Bahnhof Lehe,525,de:VBN:525,True,False,False,,False
4,2024-10-28,TN 5 CUX,None,Reisedienst von Rahden GmbH & Co. KG,1553101,Wehden Feuerwehr,Schiffdorf Schulzentrum,553,de:VBN:553,False,False,False,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5950,2024-12-20,TN 5 CUX,None,Reisedienst von Rahden GmbH & Co. KG,1553106,Spaden Grundschule,Wehden Feuerwehr,553,de:VBN:553,False,False,False,,False
5951,2024-12-20,TN 5 CUX,None,Reisedienst von Rahden GmbH & Co. KG,1542111,Ringstedt Keilstraße,Bad Bederkesa Moor-Therme,542,de:VBN:542,False,False,False,,False
5952,2024-12-20,TN 5 CUX,None,Reisedienst von Rahden GmbH & Co. KG,1553111,Wehden Feuerwehr,Schiffdorf Schulzentrum,553,de:VBN:553,False,False,False,,False
5953,2024-12-20,TN 5 CUX,None,Reisedienst von Rahden GmbH & Co. KG,1553108,Schiffdorf Schulzentrum,Wehden Feuerwehr,553,de:VBN:553,False,False,False,,False


### Häufung von Fahrten ohne Echtzeit

In [26]:
df_fahrten_ohne_ez = rt.cursor.sql("""
              
                select datum::date as datum, ebene, lineshort , fnr, hasrealtime
               
                from vw_buendel 
                where datum >= (current_date - interval 30 day) and hasrealtime = false
                group by all
                order by ebene, lineshort, fnr
    
              """).df()

df_fahrten_ohne_ez_zusatz = df_fahrten_ohne_ez.merge(df_zusatz, left_on = ['datum', 'fnr'], right_on = ['datum', 'fnr'], how='left')
df_fahrten_ohne_ez_zusatz.query("~vu.isnull()") 

df_fahrten_ohne_ez_zusatz[['lineshort_x','datum','fnr']].groupby(['lineshort_x','fnr'], as_index=False)\
    .agg(datum_min=('datum', 'min'), datum_max=('datum', 'max'), count=('datum', 'count')).sort_values('count', ascending=False)\
    .to_excel('out/rt_fahrten_ohne_ez_zusatz.xlsx', index=False)

In [27]:
df_fahrten_ohne_ez_zusatz.query("~vu.isnull()")

,datum,ebene,lineshort_x,fnr,hasRealtime,lineshort_y,lineid,vu


In [28]:
interval_auswertung = 21
df_fahrten_mit_nicht_vollstaendiger_echtzeit = rt.cursor.sql(f"""
              select * from 
                (select ebene, lineshort , fnr, count(*) as anz, count(*) filter (hasRealtime) as anz_rt, 
                (anz - anz_rt) as f_ohne_rt ,round(anz_rt/anz,2) as quote,
                max(datum::date) filter (hasRealtime) as letzte_lieferung_echtzeit
                from vw_buendel 
                where datum >= (current_date - interval {interval_auswertung} day)
                group by all
                order by ebene, lineshort, fnr)
              where f_ohne_rt > 1 and ebene in ('1+','1', '2') 
                order by f_ohne_rt desc                                                             
            
              """).df()

df_fahrten_mit_nicht_vollstaendiger_echtzeit

,ebene,lineshort,fnr,anz,anz_rt,f_ohne_rt,quote,letzte_lieferung_echtzeit


In [29]:
xl = 'out/nicht_vollstaendig.xlsx'
sn01 = '01 fahrten_rt_kl_100_roz'
sn02 = '02 zusatzfahrten'
sn03 = '03 ohne ez merge zusatz'

with pd.ExcelWriter(xl, engine='openpyxl') as writer: 
    df_fahrten_mit_nicht_vollstaendiger_echtzeit.to_excel(writer, index=False, sheet_name=sn01)
    writer.book[sn01].freeze_panes = 'A2'
    writer.book[sn01].auto_filter.ref='A:H'

    df_zusatz.to_excel(writer, index=False, sheet_name=sn02)
    writer.book[sn02].freeze_panes = 'A2'
    writer.book[sn02].auto_filter.ref='A:H'

    df_fahrten_ohne_ez_zusatz.to_excel(writer, index=False, sheet_name=sn03)
    writer.book[sn03].freeze_panes = 'A2'
    writer.book[sn03].auto_filter.ref='A:H'


In [30]:
q = rt.cursor.sql("""
                   (select 
                    datum::date as datum, ebene, lineshort, lineid_short, count(*) anz,
                    count(*) filter (hasRealtime) anz_rt, round(anz_rt/ anz,2) anteil_rt, 
                    max(datum) filter (hasRealtime) letzte_lieferung
                    from vw_buendel 
                    where datum >= date_trunc('month', (date_trunc('month',current_date) - interval 1 day)::date)
                    and datum <= (date_trunc('month',current_date) - interval 1 day)::date
                  
                    group by all

                    order by datum::date)
                  """)
#q.filter("lineshort in ('S35', '350')") #mit filter einfache Abfragen

q

┌────────────┬─────────┬───────────┬──────────────┬───────┬────────┬───────────┬─────────────────────┐
│   datum    │  ebene  │ lineshort │ lineid_short │  anz  │ anz_rt │ anteil_rt │  letzte_lieferung   │
│    date    │ varchar │  varchar  │   varchar    │ int64 │ int64  │  double   │      timestamp      │
├────────────┼─────────┼───────────┼──────────────┼───────┼────────┼───────────┼─────────────────────┤
│ 2024-12-02 │ NULL    │ 555       │ de:VBN:555   │     7 │      6 │      0.86 │ 2024-12-02 00:00:00 │
│ 2024-12-02 │ NULL    │ 529       │ de:VBN:529   │    11 │      6 │      0.55 │ 2024-12-02 00:00:00 │
│ 2024-12-02 │ NULL    │ 542       │ de:VBN:542   │    22 │      7 │      0.32 │ 2024-12-02 00:00:00 │
│ 2024-12-02 │ NULL    │ 545       │ de:VBN:545   │     8 │      3 │      0.38 │ 2024-12-02 00:00:00 │
│ 2024-12-02 │ NULL    │ 553       │ de:VBN:553   │    12 │      0 │       0.0 │ NULL                │
│ 2024-12-02 │ NULL    │ 525       │ de:VBN:525   │     2 │      0 │     

In [31]:
#Abfrage für den letzten Monat
q_pivot_lm = rt.cursor.sql("""
                    pivot (select 
                            datum::date as datum, ebene, lineshort, lineid_short, count(*) anz,
                            count(*) filter (hasRealtime) anz_rt, round(anz_rt/ anz,2) anteil_rt
                        from vw_buendel 
                        where datum >= date_trunc('month', (date_trunc('month',current_date) - interval 1 day)::date)
                            and datum <= (date_trunc('month',current_date) - interval 1 day)::date
                        group by all
                        )
                    on datum
                    using sum(anteil_rt)
                    group by lineshort, ebene
                    order by ebene, lineshort""")

q_pivot_lm.df().fillna('-')

,lineshort,ebene,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-08,2024-12-09,2024-12-10,2024-12-11,2024-12-12,2024-12-13,2024-12-16,2024-12-17,2024-12-18,2024-12-19,2024-12-20,2024-12-23,2024-12-27,2024-12-30
0,525,-,0.00,0.00,0.00,0.00,0.00,-,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,529,-,0.55,0.55,0.55,0.55,0.44,-,0.67,0.64,0.64,0.64,0.56,-,-,-,-,-,-,-,-
2,542,-,0.32,0.23,0.23,0.23,0.33,0.0,0.39,0.39,0.22,0.22,0.18,0.33,0.25,0.25,0.25,0.25,0.0,0.0,0.0
3,545,-,0.38,0.38,0.38,0.38,0.40,-,0.40,0.40,0.40,0.40,0.43,-,-,-,-,-,-,-,-
4,553,-,0.00,0.00,0.00,0.00,0.00,-,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,-,-,-
5,555,-,0.86,0.86,0.86,0.86,0.83,-,0.86,0.86,0.86,0.86,0.83,0.0,0.0,0.0,0.0,0.0,-,-,-


## Ausgabe je Bündel als html

In [32]:
list_buendel = rt.cursor.sql("select distinct buendel from linien where buendel not in ('nahsh')").df()['buendel'].to_list()

In [33]:
#Zellformatierung CSS
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal; font-family: sans-serif; font-size: 15px;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #FFFFFF; color: #000000; font-family: sans-serif; font-size: 15px;text-orientation: upright;'
}

td = {'selector' : 'td', 'props': 'text-align:right; font-family: sans-serif; font-size: 14px;'}

In [34]:
rt.cursor.sql(f"""describe fahrten""")

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ datum                       │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ fnr                         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ destination                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ hasRealtime                 │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ vu                          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid                      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid_short                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ lineshort                   │ VARCHAR     │ YES     

In [35]:
func_proz = lambda s: str(int((1-s) * 1000)/10) + '%' if str(int(s)) != '-1' else '-'
func_date = lambda s: s.dt.strftime('%m/%d/%Y')

interval_auswertung = 21

for b in list_buendel:
    print(b, b.replace(' ', '_').lower(), replace_german_special_characters(b).replace(' ', '_').lower())

    rt.create_vw_buendel(b)
    #Abfrage für die letzten 30 Tage
    q_pivot_lm = rt.cursor.sql(f"""
                        pivot (select 
                                datum::date as datum, ebene, lineshort, lineid_short, count(*) anz,
                                count(*) filter (hasRealtime) anz_rt, round(anz_rt/ anz,2) anteil_rt
                            from vw_buendel 
                            where datum >= (current_date - interval {interval_auswertung} day)
                            group by all
                            )
                        on datum
                        using sum(anteil_rt)
                        group by lineshort, ebene
                        order by ebene, lineshort""")
    
    #Liste der Fahrten ohne Echtzeit die häufiger als 1 mal vorkommen
    df_fahrten_mit_nicht_vollstaendiger_echtzeit = rt.cursor.sql(f"""
                select * from 
                    (select ebene, lineshort , fnr, count(*) as anz, count(*) filter (hasRealtime) as anz_ez, 
                    (anz - anz_ez) as fahrten_ohne_ez ,round(anz_ez/anz,2) as quote,
                    max(datum::date) filter (hasRealtime) as letzte_lieferung_echtzeit
                    from vw_buendel 
                    where datum >= (current_date - interval {interval_auswertung} day)
                    group by all
                    order by ebene, lineshort, fnr)
                where fahrten_ohne_ez > 1 and ebene in ('1+','1', '2','Nacht') 
                    order by fahrten_ohne_ez desc                                                             
                
                """).df()
    
    df_fahrten_ohne_ez = rt.cursor.sql(f"""              
                select datum::date as datum, ebene, lineshort , fnr, hasrealtime               
                from vw_buendel 
                where datum >= (current_date - interval {interval_auswertung} day) and hasrealtime = false
                group by all
                order by ebene, lineshort, fnr
    
              """).df()
    
    df_fahrten_gesamt = rt.cursor.sql(f"""              
                select *               
                from vw_buendel 
                where datum >= (current_date - interval {interval_auswertung} day) 
                order by ebene, lineshort, fnr
    
              """).df()
    
    html_zusatz_table = 'html/pre_zusatz.html'
    df_fahrten_ohne_ez_zusatz = df_fahrten_ohne_ez.merge(df_zusatz, left_on = ['datum', 'fnr'], right_on = ['datum', 'fnr'], how='left')
    df_fahrten_ohne_ez_zusatz.query("~vu.isnull()").to_html(html_zusatz_table, index=False)

    html_pre_table = 'html/pre_table.html'
    df_fahrten_mit_nicht_vollstaendiger_echtzeit.to_html(html_pre_table, index=False)

    html_pre_pivot = 'html/pre_pivot.html'
    q_pivot_lm.df().style.background_gradient(cmap="RdYlGn", axis = None,  vmin=0.5, vmax=1).highlight_null(color='white')\
        .format( precision=2, na_rep='-', thousands=" ")\
        .highlight_null(color='white')\
        .set_table_styles([index_names, headers, td])\
        .to_html(html_pre_pivot)
    
    # Save the HTML table to a file (optional)   
    with open(html_pre_pivot, 'r') as file:
        html_pre_pivot = file.read()
    
    # Load the HTML page template
    with open('html/template.html', 'r') as file:
        html_template = file.read()

    # Insert the HTML table into the template
    title = f"Echtzeitquote Bündel {b} je Linie erstellt: {dt.datetime.now().strftime('%d.%m.%Y %H:%M')}" 
    html_page = html_template.replace('{{ html_pivot }}', html_pre_pivot).replace('{{ html_title }}', title)

    if df_fahrten_mit_nicht_vollstaendiger_echtzeit.shape[0] > 0:
        html_page = html_page.replace('{{ html_table }}', df_fahrten_mit_nicht_vollstaendiger_echtzeit.to_html(index=False))
    else:
        html_page = html_page.replace('{{ html_table }}', "Keine Häufung Fahrten ohne Echtzeit")

    if df_fahrten_ohne_ez_zusatz.query("~vu.isnull()").shape[0] > 0:
        html_page = html_page.replace('{{ html_table_zusatz }}', df_fahrten_ohne_ez_zusatz.query("~vu.isnull()").to_html(index=False))
    else:
        html_page = html_page.replace('{{ html_table_zusatz }}', "Keine Zusatzfahrten mit gleicher Fahrtnummer")

    # Save the combined HTML page to a file
    html_combined = f"/var/www/rt_archiv/buendel/rt_{replace_german_special_characters(b).replace(' ', '_').lower()}.html"


    with open(html_combined, 'w') as file:
        file.write(html_page)


    #Ausgabe der wichtigen Ergebnisse als Excel
    xl = f"buendel_stat/{replace_german_special_characters(b).replace(' ', '_').lower()}_stat.xlsx"

    sn00 = '01 hilfe'
    sn02 = '02 statistik'
    sn03 = '03 fahrten gesamt'  

    with pd.ExcelWriter(xl, engine='openpyxl') as writer:         
        q_pivot_lm.df().to_excel(writer, index=True, sheet_name=sn02)
        writer.book[sn02].freeze_panes = 'A2'
        writer.book[sn02].auto_filter.ref='A:H'

        df_fahrten_gesamt.to_excel(writer, index=False, sheet_name=sn03)
        writer.book[sn03].freeze_panes = 'A2'
        writer.book[sn03].auto_filter.ref='A:N'

    # Öffnen des Workbooks und Anwenden der Formatierung
    wb = openpyxl.load_workbook(xl)

    #Erstellen des Hilfeblattes an erster Position
    wb.create_sheet(sn00, index=0)
    sheet = wb[sn00]
    sheet['A1'] = f"Erstellt: {dt.datetime.now().strftime('%Y-%m-%d %H:%M')}"
    sheet['A2'] = f"Erläuterung der Werte in der Tabelle"
    sheet['A3'] = f"Blatt {sn02} enthält die Echtzeitquote der Linien des Bündels {b} für die letzten {interval_auswertung} Tage"

    wb.save(xl)

       

WM Nord wm_nord wm_nord
DH Südost dh_südost dh_suedost
OL Südost ol_südost ol_suedost
OL West ol_west ol_west
VER Ost ver_ost ver_ost
AM Süd am_süd am_sued
BHV bhv bhv
OL Nord ol_nord ol_nord
RBVBN rbvbn rbvbn
VER Südwest ver_südwest ver_suedwest
OHZ Mitte ohz_mitte ohz_mitte
OHZ West ohz_west ohz_west
REKREUZ rekreuz rekreuz
VER Nord ver_nord ver_nord
WM Süd wm_süd wm_sued
AM West am_west am_west
OHZ Ost ohz_ost ohz_ost
OL Stadt ol_stadt ol_stadt
RSB rsb rsb
TN 5 CUX tn_5_cux tn_5_cux
AM Ost am_ost am_ost
DH Nordwest dh_nordwest dh_nordwest
DH Südwest dh_südwest dh_suedwest
HB Stadt hb_stadt hb_stadt
HB Straßenbahn hb_straßenbahn hb_strassenbahn
DEL del del
DH Nordost dh_nordost dh_nordost


In [36]:
q_pivot_lm.df()

,lineshort,ebene,2024-12-19,2024-12-20,2024-12-21,2024-12-22,2024-12-23,2024-12-24,2024-12-25,2024-12-26,2024-12-27,2024-12-28,2024-12-29,2024-12-30,2024-12-31,2025-01-01,2025-01-02,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-07,2025-01-08
0,150,1+,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00
1,152,2,1.0,1.00,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.00
2,177,2,1.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.00
3,151,3,1.0,0.95,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.00
4,153,3,1.0,0.83,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.00
5,154,3,1.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.00
6,164,3,1.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.00
7,165,3,1.0,0.95,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,0.92
8,178,3,1.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.00
9,179,3,1.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.00


In [37]:
rt.cursor.sql(f"""
                select * from 
                    (select ebene, lineshort , fnr, count(*) as anz, count(*) filter (hasRealtime) as anz_ez, 
                    (anz - anz_ez) as fahrten_ohne_ez ,round(anz_ez/anz,2) as quote,
                    max(datum::date) filter (hasRealtime) as letzte_lieferung_echtzeit
                    from vw_buendel 
                    where datum >= (current_date - interval {interval_auswertung} day)
                    group by all
                    order by ebene, lineshort, fnr)
                where fahrten_ohne_ez > 1 and ebene in ('1+','1', '2','Nacht') 
                    order by fahrten_ohne_ez desc                                                             
                
                """).df().shape[0]

0

In [38]:
html_template

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Echtzeitauswertung Bündel</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            margin: 0;\n            padding: 0;\n            background-color: #ffffff;\n            color: #333;\n            font-size: 12px;\n        }\n        table {\n            width: 100%;\n            border-collapse: collapse;\n        }\n        table, th, td {\n            border: 1px solid white;\n            \n            margin-left: 0; /* Align the table to the left */\n            margin-right: auto; /* Align the table to the left */\n        }\n        th, td {\n            padding: 4px;\n            text-align: right;\n            font-family: Arial, sans-serif;\n        }\n        h2 {\n            font-family: Arial, sans-serif;\n        }\n    </style>\n</head>\n<body>\n    <h2>{{ html_title }}</h2>\

In [39]:
rt.anzahl_fahrten_betreiber().df()

AttributeError: 'DataFrame' object has no attribute 'df'

In [34]:
logging.info(f"Anzahl Fahrten gesamt {rt.anzahl_fahrten()}")

## Ohne class

In [35]:
con = duckdb.connect()

In [36]:
con.sql(f"""INSTALL postgres;
LOAD postgres;
ATTACH 'dbname=zvbn_postgis user={config['POSTGRES_USER']} host=127.0.0.1 password={config['POSTGRES_PW']}' AS db_dm (TYPE POSTGRES, READ_ONLY);""")

In [ ]:
con.sql("create or replace table lin_buendel as select * from db_dm.basis.lin_buendel")
con.sql("select * from lin_buendel")

┌─────────────────────┬────────────┬───────────────────────┬───────┬───────────────────────────────────────────┬────────────┬────────┬─────────┬─────────────────────┬────────────────────────────────────────────────────────────────┬─────────────┬───────────────────┐
│       buendel       │   harmon   │         kreis         │  id   │                 bemerkung                 │ kreis_kurz │ lfd_nr │  aktiv  │    buendel_lang     │                          project_vms                           │ rt_operator │  betreiber_kurz   │
│       varchar       │    date    │        varchar        │ int32 │                  varchar                  │  varchar   │ int32  │ boolean │       varchar       │                            varchar                             │   varchar   │      varchar      │
├─────────────────────┼────────────┼───────────────────────┼───────┼───────────────────────────────────────────┼────────────┼────────┼─────────┼─────────────────────┼────────────────────────────────────

In [ ]:
sql_lin = """
        Create or replace table linien as 
        SELECT nummer AS linie, buendel, ebene, dlid, id 
        FROM db_dm.basis.linien 
        WHERE buendel IS NOT NULL AND aktiv IS TRUE 
        ORDER BY buendel, ebene, nummer """
con.sql(sql_lin)
con.sql("select * from linien")

┌─────────┬─────────┬─────────┬───────────────┬───────┐
│  linie  │ buendel │  ebene  │     dlid      │  id   │
│ varchar │ varchar │ varchar │    varchar    │ int32 │
├─────────┼─────────┼─────────┼───────────────┼───────┤
│ 330     │ AM Ost  │ 1+      │ de:VBN:330    │     7 │
│ 340     │ AM Ost  │ 1+      │ de:VBN:340    │   348 │
│ 370     │ AM Ost  │ 2       │ de:VBN:370    │    59 │
│ 331     │ AM Ost  │ 3       │ de:VBN:331    │   287 │
│ 332     │ AM Ost  │ 3       │ de:VBN:332    │   231 │
│ 333     │ AM Ost  │ 3       │ de:VBN:333    │   232 │
│ 334     │ AM Ost  │ 3       │ de:VBN:334    │   233 │
│ 335     │ AM Ost  │ 3       │ de:VBN:335    │   234 │
│ 336     │ AM Ost  │ 3       │ de:VBN:336    │   235 │
│ 337     │ AM Ost  │ 3       │ de:VBN:337    │   360 │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│ 448     │ WM Süd  │ 3       │ de:VBN:448    │ 

### Abruf der Parquet Files (Tagespakete)

In [39]:
server = 'prod'
con.sql(f"create or replace table fahrten as select * from read_parquet('out/parquet/{server}/fahrten*.parquet',  union_by_name = true, filename = true)")
con.sql(f"create or replace table verlauf as select * from read_parquet('out/parquet/{server}/verlauf*.parquet',  union_by_name = true, filename = true)")
con.sql(f"create or replace table zusatz as select * from read_parquet('out/parquet/{server}/zusatz*.parquet',  union_by_name = true, filename = true)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Ermitteln und Löschen von nicht gewollten Betreibern

In [40]:
#con.sql("select distinct vu from fahrten where vu like '%Weser%'")

In [41]:
con.sql("describe fahrten")

┌─────────────────────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type  │  null   │   key   │ default │  extra  │
│           varchar           │   varchar    │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ datum                       │ TIMESTAMP_NS │ YES     │ NULL    │ NULL    │ NULL    │
│ fnr                         │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ destination                 │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ hasRealtime                 │ BOOLEAN      │ YES     │ NULL    │ NULL    │ NULL    │
│ vu                          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid                      │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid_short                │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineshort                   │ VARCHAR    

In [ ]:
con.sql("select count(*), datum from fahrten group by datum order by datum")

┌──────────────┬─────────────────────┐
│ count_star() │        datum        │
│    int64     │    timestamp_ns     │
├──────────────┼─────────────────────┤
│        17666 │ 2024-08-29 00:00:00 │
│        17378 │ 2024-08-30 00:00:00 │
│        10362 │ 2024-08-31 00:00:00 │
│         6947 │ 2024-09-01 00:00:00 │
│        17593 │ 2024-09-02 00:00:00 │
│        17625 │ 2024-09-03 00:00:00 │
│        17618 │ 2024-09-04 00:00:00 │
│        17613 │ 2024-09-05 00:00:00 │
│        17410 │ 2024-09-06 00:00:00 │
│        10638 │ 2024-09-07 00:00:00 │
│          ·   │          ·          │
│          ·   │          ·          │
│          ·   │          ·          │
│        14327 │ 2024-12-27 00:00:00 │
│        10632 │ 2024-12-28 00:00:00 │
│         7011 │ 2024-12-29 00:00:00 │
│        14182 │ 2024-12-30 00:00:00 │
│         9698 │ 2024-12-31 00:00:00 │
│         7327 │ 2025-01-01 00:00:00 │
│        14195 │ 2025-01-02 00:00:00 │
│        14312 │ 2025-01-03 00:00:00 │
│        10671 │ 2025-01-

In [43]:
if False: #True / False um ggf. weiterhin alles durchlaufen zu lassen
    print('Löschen von Betreibern')
    con.sql("delete from fahrten where vu not in ('Weser-Ems-Bus Betrieb Bremen', 'Weser-Ems-Bus Auftragnehmerleistungen')")
    #con.sql("delete from verlauf where vu not in ('Weser-Ems-Bus Betrieb Bremen', 'Weser-Ems-Bus Auftragnehmerleistungen')")
    #con.sql("delete from zusatz where vu not in ('Weser-Ems-Bus Betrieb Bremen', 'Weser-Ems-Bus Auftragnehmerleistungen')")

In [44]:
con.sql(f"select count(*) from fahrten where datum >= (current_date - interval 100 days)").df().values.tolist()[0][0]

1378108

In [ ]:
anzahl_fahrten = con.sql(f"select count(*) from fahrten where datum >= '{letzte14tage}'").df().values.tolist()[0][0]
print(f"""Anzahl Fahrten: {anzahl_fahrten},  Länge Verlauf: {con.sql("select count(*) from verlauf").df().values.tolist()[0][0]}    """) 

Anzahl Fahrten: 146105,  Länge Verlauf: 43604836    


In [46]:
con.sql("""select 
            datum, 
            fahrtstartstationname, 
           strftime( cast(fahrtstarttime as TIMESTAMPTZ), '%H:%M') as fahrtstart,
           fahrtendstationname,
           strftime( cast(fahrtendtime as TIMESTAMPTZ), '%H:%M') as fahrtende,
            
            deviceid, 
            split_part(deviceid, '-', 2) as fnr, 
            cast(((cast(split_part(split_part(deviceid, '-', 3), '#', 1) as int64) - 8000000000000) / 1000) as int64) as m2, 
        from fahrten 
        where deviceid like '%680%DBRB%' and datum = '2024-10-29'
        order by datum, fahrtstarttime
        
        """).df()
#.to_excel('out/web.xlsx', index=False)

,datum,fahrtstartstationname,fahrtstart,fahrtendstationname,fahrtende,deviceid,fnr,m2
0,2024-10-29,Wallhöfen Am Mühlenberg,04:08,Bremen Gröpelingen,05:36,1029-1680002-8014352100000#!ADD!#DBRB#,1680002,14352100
1,2024-10-29,Bremen Gröpelingen,05:08,Wallhöfen Am Mühlenberg,06:32,1029-1680001-8014349300000#!ADD!#DBRB#,1680001,14349300
2,2024-10-29,Wallhöfen Am Mühlenberg,05:24,Bremen Gröpelingen,06:52,1029-1680004-8014352101000#!ADD!#DBRB#,1680004,14352101
3,2024-10-29,Bremen Gröpelingen,06:08,Wallhöfen Am Mühlenberg,07:32,1029-1680003-8014349400000#!ADD!#DBRB#,1680003,14349400
4,2024-10-29,Wallhöfen Am Mühlenberg,06:46,Bremen Gröpelingen,08:26,1029-1680006-8014354100000#!ADD!#DBRB#,1680006,14354100
5,2024-10-29,Bremen-Burg(Bus),06:53,Osterholz-Scharmbeck Bahnhof (Bus),07:40,1029-1680005-8014351300000#!ADD!#DBRB#,1680005,14351300
6,2024-10-29,Ritterhude Struckbergstraße,06:55,Osterholz-Scharmbeck Bahnhof (Bus),07:37,1029-1680905-8014351500000#!ADD!#DBRB#,1680905,14351500
7,2024-10-29,Wallhöfen Am Mühlenberg,07:46,Bremen Gröpelingen,09:21,1029-1680008-8014354200000#!ADD!#DBRB#,1680008,14354200
8,2024-10-29,Bremen Gröpelingen,08:08,Wallhöfen Am Mühlenberg,09:32,1029-1680007-8014349200000#!ADD!#DBRB#,1680007,14349200
9,2024-10-29,Wallhöfen Am Mühlenberg,08:24,Bremen Gröpelingen,09:52,1029-1680010-8014352200000#!ADD!#DBRB#,1680010,14352200


### Anzahl der Fahrten je Betreiber

In [47]:
con.sql("select journeyOperator, count(journeyOperator) as count from verlauf group by journeyOperator order by count")

┌─────────────────────────────────────────────────────────────┬──────────┐
│                       journeyOperator                       │  count   │
│                           varchar                           │  int64   │
├─────────────────────────────────────────────────────────────┼──────────┤
│ Autobus Stoss GmbH                                          │    33484 │
│ SBV Janßen GmbH & Co. KG                                    │    53635 │
│ MEW Mobilitätszentrale Elbe-Weser                           │    66553 │
│ WestfalenBahn                                               │    79926 │
│ eurobahn                                                    │   105137 │
│ Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH(Bahndaten) │   106022 │
│ metronom Eisenbahngesellschaft mbH                          │   109361 │
│ Regionalverkehre Start Deutschland (Niedersachsen-Mitte)    │   113629 │
│ Weser-Ems-Bus Auftragnehmerleistungen                       │   179524 │
│ W. Giese Nachf. Omnibus

### Fahrten mit hohen Verspätungen

In [48]:
con.sql("select distinct deviceid from verlauf where dep_del > 100").df()

,deviceid
0,0904-1740027-00027-1#!ADD!#BSAG#
1,0923-735001-00001-1#!ADD!#BSAG#
2,1025-5731031-00031-1#!ADD!#BSAG#
3,1025-1330065#!ADD!#IVU-Regio#
4,1028-1347004#!ADD!#IVU-Regio#
...,...
1391,1202-3259015-00015-1#!ADD!#BSAG#
1392,1213-640005-00005-1#!ADD!#BSAG#
1393,1223-1347008#!ADD!#IVU-Regio#
1394,0102-1347004#!ADD!#IVU-Regio#


In [ ]:
con.sql("describe fahrten")

┌─────────────────────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type  │  null   │   key   │ default │  extra  │
│           varchar           │   varchar    │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ datum                       │ TIMESTAMP_NS │ YES     │ NULL    │ NULL    │ NULL    │
│ fnr                         │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ destination                 │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ hasRealtime                 │ BOOLEAN      │ YES     │ NULL    │ NULL    │ NULL    │
│ vu                          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid                      │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid_short                │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineshort                   │ VARCHAR    

### Verkürzung der DLID
- Zum Teil weren bei mehreren Betreibern einer Linie TLID mit vierteiliger DLID geliefert 
- Verkürzung ermöglicht die Verknüpfung mit Liste aus DM

In [50]:
con.sql("alter table fahrten add column if not exists lineid_short VARCHAR")
con.sql("""update fahrten 
        set lineid_short = concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3))""")
con.sql("""select distinct lineid, 
        concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3)) 
        from fahrten""")

┌───────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────┐
│    lineid     │ concat_ws(':', split_part(lineid, ':', 1), split_part(lineid, ':', 2), split_part(lineid, ':', 3)) │
│    varchar    │                                              varchar                                               │
├───────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ de:VBN:N40:   │ de:VBN:N40                                                                                         │
│ de:VBN:N41:   │ de:VBN:N41                                                                                         │
│ de:VBN:RE1:   │ de:VBN:RE1                                                                                         │
│ de:VBN:329:1  │ de:VBN:329                                                                                         │
│ de:VBN:95:    │ de:VBN:95                     

### Über HIM gemeldete Ausfälle (ts_reported_cancelled gefüllt)

In [51]:
df_fahrten_ausfall_him = con.sql(f"""
                              select vu, fnr, ts_reported_cancelled, journey_cancelled 
                              from fahrten f 
                              where ts_reported_cancelled != '' and f.datum >= '{letzte14tage}'""").df()

### Echzeitquote

#### nach Linie und Betreiber

In [52]:
df_ez_quote_betreiber = con.sql(f"""
        select l.buendel, l.ebene,f.datum, f.vu, f.lineshort,f.lineid_short, count(f.hasRealtime) filter (f.hasRealtime = True) ez_true, count(f.*) count, 
        round(ez_true / count * 100, 1) anteil_ez
        from fahrten f
        left outer join linien l on f.lineid_short = l.dlid
        where f.datum >= '{letzte14tage}'              
        group by f.lineid_short, f.vu, f.datum, f.lineshort, f.lineid_short, l.buendel, l.ebene
        order by f.vu, f.lineid_short
        """).df()
df_ez_quote_betreiber['buendel'] = df_ez_quote_betreiber['buendel'].fillna('-')
df_ez_quote_betreiber['ebene'] = df_ez_quote_betreiber['ebene'].fillna('-')
anteil_ez_pivot_betreiber = pd.pivot_table(df_ez_quote_betreiber, index=['buendel','ebene', 'vu', 'lineshort'], columns='datum', values='anteil_ez').reset_index()
anteil_ez_pivot_betreiber

datum,buendel,ebene,vu,lineshort,2024-12-23 00:00:00,2024-12-24 00:00:00,2024-12-25 00:00:00,2024-12-26 00:00:00,2024-12-27 00:00:00,2024-12-28 00:00:00,2024-12-29 00:00:00,2024-12-30 00:00:00,2024-12-31 00:00:00,2025-01-01 00:00:00,2025-01-02 00:00:00,2025-01-03 00:00:00,2025-01-04 00:00:00,2025-01-05 00:00:00
0,-,-,Autobus Stoss GmbH,559,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
1,-,-,BREMERHAVEN BUS,518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-,-,BREMERHAVEN BUS,519,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN
3,-,-,BREMERHAVEN BUS,S,100.0,NaN,NaN,NaN,100.0,NaN,NaN,100.0,NaN,NaN,100.0,100.0,NaN,NaN
4,-,-,Bremer Straßenbahn AG,10E,100.0,NaN,NaN,NaN,100.0,NaN,NaN,100.0,NaN,NaN,100.0,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,WM Süd,1,Gebken Reisen GmbH,440,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,96.9,100.0,100.0,100.0,100.0,100.0
372,WM Süd,1,Gebken Reisen GmbH,450,93.8,100.0,100.0,100.0,100.0,100.0,100.0,96.9,100.0,100.0,87.5,96.9,100.0,100.0
373,WM Süd,2,Gebken Reisen GmbH,460,100.0,91.7,NaN,NaN,93.8,100.0,NaN,100.0,91.7,NaN,100.0,100.0,100.0,NaN
374,WM Süd,3,Gebken Reisen GmbH,431,0.0,NaN,NaN,NaN,14.3,NaN,NaN,42.9,NaN,NaN,28.6,28.6,NaN,NaN


#### nach Linie (ohne Betreiber)

In [53]:
df_ez_quote_o_betreiber = con.sql(f"""
        select l.buendel, l.ebene,f.datum, f.lineshort,f.lineid_short, count(f.hasRealtime) filter (f.hasRealtime = True) ez_true, count(f.*) count, 
        round(ez_true / count * 100, 1) anteil_ez
        from fahrten f        
        left outer join linien l on f.lineid_short = l.dlid      
        where f.datum >= '{letzte14tage}'        
        group by f.lineid_short, f.datum, f.lineshort, f.lineid_short, l.buendel, l.ebene
        order by f.lineid_short
        """).df()
df_ez_quote_o_betreiber['buendel'] = df_ez_quote_o_betreiber['buendel'].fillna('-')
df_ez_quote_o_betreiber['ebene'] = df_ez_quote_o_betreiber['ebene'].fillna('-')
anteil_ez_pivot_o_betreiber = pd.pivot_table(df_ez_quote_o_betreiber, index=['buendel','ebene', 'lineshort'], columns='datum', values='anteil_ez').reset_index()
anteil_ez_pivot_o_betreiber

datum,buendel,ebene,lineshort,2024-12-23 00:00:00,2024-12-24 00:00:00,2024-12-25 00:00:00,2024-12-26 00:00:00,2024-12-27 00:00:00,2024-12-28 00:00:00,2024-12-29 00:00:00,2024-12-30 00:00:00,2024-12-31 00:00:00,2025-01-01 00:00:00,2025-01-02 00:00:00,2025-01-03 00:00:00,2025-01-04 00:00:00,2025-01-05 00:00:00
0,-,-,1020,100.0,100.0,NaN,NaN,100.0,100.0,NaN,100.0,100.0,NaN,100.0,100.0,50.0,NaN
1,-,-,1021,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,-,-,10E,100.0,NaN,NaN,NaN,100.0,NaN,NaN,100.0,NaN,NaN,100.0,100.0,NaN,NaN
3,-,-,14,100.0,100.0,100.0,100.0,100.0,100.0,98.3,99.1,100.0,100.0,100.0,100.0,100.0,100.0
4,-,-,1E,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,94.4,100.0,100.0,100.0,95.5,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,WM Süd,1,440,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,96.9,100.0,100.0,100.0,100.0,100.0
355,WM Süd,1,450,93.8,100.0,100.0,100.0,100.0,100.0,100.0,96.9,100.0,100.0,87.5,96.9,100.0,100.0
356,WM Süd,2,460,100.0,91.7,NaN,NaN,93.8,100.0,NaN,100.0,91.7,NaN,100.0,100.0,100.0,NaN
357,WM Süd,3,431,0.0,NaN,NaN,NaN,14.3,NaN,NaN,42.9,NaN,NaN,28.6,28.6,NaN,NaN


### Fahrten ohne Echtzeit Ebene 1/1+ und 2

In [54]:
df_fahrten_ohne_ez_ebenen_1_1p_2 = con.sql(f"""
        select f.datum, l.buendel, l.ebene, f.vu, f.fnr, f.lineshort,f.lineid_short, f.hasrealtime, f.journey_cancelled, f.reported_cancelled, f.ts_reported_cancelled
        
        from fahrten f
                                           
        left outer join linien l on f.lineid_short = l.dlid              
        where l.ebene in ('1', '1+') and f.hasrealtime = False and f.datum >= '{letzte14tage}'
                                           
        order by f.datum, f.lineid_short
        """).df()

In [55]:
df_fahrten_ausfall_1_1p_2 = con.sql(f"""
        select f.datum, l.buendel, l.ebene, f.vu, f.fnr, f.lineshort,f.lineid_short, f.hasrealtime, f.journey_cancelled, f.reported_cancelled, f.ts_reported_cancelled
        
        from fahrten f
                                    
        left outer join linien l on f.lineid_short = l.dlid              
        where l.ebene in ('1', '1+', '2') and (journey_cancelled = True or f.reported_cancelled = True) and 
        f.datum >= '{letzte14tage}'                            
        order by f.datum, f.lineid_short
        """).df()

# Ausgabe xlsx EZ Statistiken

In [56]:
xlsx = "/var/www/rt_archiv/anteil_echtzeit_linien_vbn.xlsx"
sn00 = '00 Hilfe'
sn01 = '01 pivot alle Linien betreiber'
sn02 = '02 pivot alle Linien'
sn03 = '03 fahrten ohne EZ 1 1+ 2'
sn04 = '04 fahrten ohne EZ 1 1+ 3 grup'
sn06 = '05 fahrten ausfall'
sn07 = '06 fahrten ausfall über HIM'
with pd.ExcelWriter(xlsx, engine="openpyxl") as writer:
    #Hilfeblatt
    writer.book.create_sheet(sn00)
    sheet = writer.book[sn00]
    sheet['A1'] = f"Erstellt: {dt.datetime.now().strftime('%Y-%m-%d %H:%M')} Zeitraum: {letzte14tage} bis {gestern}"

    sheet['A3'] = "Inhalt"
    sheet['B4'] = f"Blatt {sn01}: Pivot Echtzeitquote inkl. Betreiberkennung"
    sheet['B5'] = f"Blatt {sn02}: Pivot Echtzeitquote ohne Betreiberkennung"
    sheet['B6'] = f"Blatt {sn03}: Fahrten ohne Echtzeit"
    sheet['B7'] = f"Blatt {sn04}: Fahrten ohne Echtzeit mit Anzahl"
    sheet['B8'] = f"Blatt {sn06}: Fahrten Ausfall"
    sheet['B9'] = f"Blatt {sn07}: Fahrten Ausfall über HIM"

    #mit Kennung der Betreiber
    anteil_ez_pivot_betreiber.to_excel(writer, sheet_name=sn01, index=False)
    writer.book[sn01].freeze_panes = 'e2'
    writer.book[sn01].auto_filter.ref='A:H'
    for cell in writer.book[sn01]["1:1"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn01].column_dimensions['c'].width = 22
    for c in ['D', 'E', 'F', 'G', 'H']:
        writer.book[sn01].column_dimensions[c].width = 22        
    for c in writer.book[sn01].iter_cols(min_col=4, max_col=anteil_ez_pivot_betreiber.shape[1]+4):
                #ermitteln der Spalte column letter
                cl = c[int(f"{anteil_ez_pivot_betreiber.shape[0]}")].column_letter
                writer.book[sn01].column_dimensions[cl].width = 16

    #Anteil EZ ohne Kennung der Betreiber
    anteil_ez_pivot_o_betreiber.to_excel(writer, sheet_name=sn02, index=False)
    writer.book[sn02].freeze_panes = 'd2'
    writer.book[sn02].auto_filter.ref='A:H'
    for cell in writer.book[sn02]["1:1"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn02].column_dimensions['c'].width = 22
    for c in ['D', 'E', 'F', 'G', 'H']:
        writer.book[sn02].column_dimensions[c].width = 22 
         
    for c in writer.book[sn02].iter_cols(min_col=4, max_col=anteil_ez_pivot_o_betreiber.shape[1]+4):
                #ermitteln der Spalte column letter
                cl = c[int(f"{anteil_ez_pivot_o_betreiber.shape[0]}")].column_letter
                writer.book[sn02].column_dimensions[cl].width = 16

    ## Ausgabe der Fahrten ohne Echtzeit Ebene 1 und 1+ und 2 einzeln
    df_fahrten_ohne_ez_ebenen_1_1p_2.to_excel(writer, sheet_name=sn03, index=False)
    writer.book[sn03].freeze_panes = 'a2'
    writer.book[sn03].auto_filter.ref='A:M'
    for cell in writer.book[sn03]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn03].column_dimensions['A'].width = 18

    ## Ausgabe der Fahrten ohne Echtzeit Ebene 1 und 1+ und 2 gruppiert mit Anzahl
    df_fahrten_ohne_ez_ebenen_1_1p_2[['vu', 'fnr']].value_counts().reset_index().sort_values(['count', 'vu'], ascending=False).to_excel(writer, sheet_name=sn04, index=False)
    writer.book[sn04].freeze_panes = 'a2'
    writer.book[sn04].auto_filter.ref='A:H'
    writer.book[sn04].column_dimensions['A'].width = 22   

    ## Ausgabe der Fahrten Ausfall Ebene 1, 1+ und 2
    df_fahrten_ausfall_1_1p_2.to_excel(writer, sheet_name=sn06, index=False)
    writer.book[sn06].freeze_panes = 'a2'
    writer.book[sn06].auto_filter.ref='A:M'
    for cell in writer.book[sn06]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn06].column_dimensions['A'].width = 18

    ## Ausgabe der Fahrten Ausfall über HIM
    df_fahrten_ausfall_him.to_excel(writer, sheet_name=sn07, index=False)
    writer.book[sn07].freeze_panes = 'a2'
    writer.book[sn07].auto_filter.ref='A:M'
    for cell in writer.book[sn07]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn07].column_dimensions['A'].width = 18

In [57]:
df_stat_rt_canc = con.sql("""
        select 
            vu, 
            count(*) as anzahl, 
            count(*) filter (hasRealtime) as hasRealtime, 
            count(*) filter (realtimeHasEverBeenReported) as realtimeHasEverBeenReported,
            count(*) filter (realtimehaseverbeenreported or hasrealtime) as rt_combined,
            count(*) filter (journey_cancelled) as journey_cancelled,
            count(*) filter (reported_cancelled) as reported_cancelled
        from fahrten
        where datum >= (current_date - interval 3 days)
        group by all
        order by vu""").df()
df_stat_rt_canc

,vu,anzahl,hasRealtime,realtimeHasEverBeenReported,rt_combined,journey_cancelled,reported_cancelled
0,AM Bus,419,400,400,400,0,0
1,AllerBus,326,287,287,287,0,0
2,Autobus Stoss GmbH,6,0,0,0,0,0
3,BREMERHAVEN BUS,3023,1752,1754,1754,2,2
4,Bremer Straßenbahn AG,12901,12626,12649,12649,7,33
5,Bruns Reisen GmbH Bad Zwischenahn,120,6,6,6,0,0
6,DB Regio AG Nord,589,581,588,588,6,6
7,Delbus GmbH & Co. KG,1066,1022,1022,1022,0,0
8,Delmenhorst-Harpstedter Eisenbahn GmbH,117,96,97,97,1,1
9,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,513,217,232,232,0,0


In [58]:
output_file = 'reports/df_stat_rt_canc.xlsx'
sheet_name = 'Stat RT Canc'
df_stat_rt_canc = df_stat_rt_canc.sort_values(by='vu')
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_stat_rt_canc.to_excel(writer, index=False, sheet_name=sheet_name)
    worksheet = writer.book[sheet_name]
    worksheet.auto_filter.ref = worksheet.dimensions
    worksheet.column_dimensions['A'].width = 30  # Set the width of column A to 30
    worksheet.freeze_panes = 'A2'  # Freeze the first row
    len = df_stat_rt_canc.shape[0]  # Get the number of rows
    worksheet[f'B{len+3}'] = f'=subtotal(9,B2:B{len + 1})'  # Add a sum formula for column B
    worksheet[f'C{len+3}'] = f'=subtotal(9,C2:C{len + 1})'  # Add a sum formula for column C
    worksheet[f'D{len+3}'] = f'=subtotal(9,D2:D{len + 1})'  # Add a sum formula for column D
    worksheet[f'E{len+3}'] = f'=subtotal(9,E2:E{len + 1})'  # Add a sum formula for column E
    worksheet[f'F{len+3}'] = f'=subtotal(9,F2:F{len + 1})'  # Add a sum formula for column F
